# Development of database code

The code in this file is not used directly in the end solution, but large sections are copied into the .py files in the /src folder.

I think Notebooks are a good way to prototype and iterate on code, particularly where it involves data (so you can visualise what's going on and prevent headaches down the line) or processes you're not completely familiar with (so you can try things rapidly until they work). This is also a good place to include extra comments along the way, which may be covered in a blog post or simply used by someone who wants to understand the code better.

The main purpose of this file is to be able to 
* save data from pandas dataframes into a sqlite database
* ensure that the process is idempotent (so you can run it multiple times and not end up with duplicates in your data, particularly historical data)
* extract data from the database and convert it to a pandas dataframe ready for use by the reinforcement learning model

Note that the docstrings in the code below were created before I went down the [PEP 257](https://peps.python.org/pep-0257/) rabbit hole, and have been retained as a reminder of this journey. See the docstrings in the src .py files for the new and improved versions, including:
* Use of """ rather than ''' for both single and multi line docstrings
* Placing the first line on the second line for multi line docstrings and indenting to the """ level (rather than having the second line be in line with the first character of the first line placed after the ''' - see examples of not what to do below)
* Using Args and Returns sections (tending towards Google-style docstring vs Numpy or others)
* Using present tense imperative (e.g. "Do this" vs "Does this")

In [1]:
# Jupyter notebooks run in a different environment so don't recognise src package by default
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
print(sys.path)

['C:\\Users\\Admin\\AppData\\Local\\Programs\\Python\\Python310\\python310.zip', 'C:\\Users\\Admin\\AppData\\Local\\Programs\\Python\\Python310\\DLLs', 'C:\\Users\\Admin\\AppData\\Local\\Programs\\Python\\Python310\\lib', 'C:\\Users\\Admin\\AppData\\Local\\Programs\\Python\\Python310', 'c:\\.venv\\happy_plants', '', 'c:\\.venv\\happy_plants\\lib\\site-packages', 'c:\\.venv\\happy_plants\\lib\\site-packages\\win32', 'c:\\.venv\\happy_plants\\lib\\site-packages\\win32\\lib', 'c:\\.venv\\happy_plants\\lib\\site-packages\\Pythonwin', 'd:\\Projects\\happy-plants']


In [2]:
import sqlite3
import pandas as pd

### Overview of SQLite with Python

From the docs:
SQLite is a C library that provides a lightweight disk-based database that doesn’t require a separate server process and allows accessing the database using a nonstandard variant of the SQL query language. Some applications can use SQLite for internal data storage. It’s also possible to prototype an application using SQLite and then port the code to a larger database such as PostgreSQL
https://docs.python.org/3/library/sqlite3.html

From ChatGPT:
SQLite is a lightweight, self-contained database engine that doesn’t require setting up a separate server. It stores data in a single .db file and has no user management or access control beyond file permissions. This makes it perfect for local applications, prototypes, and small-scale data storage.
* No server required – SQLite runs in-process with your application.
* No user/security setup – Authentication and user management aren’t built-in. Security relies on OS-level file permissions.
* Requires only the sqlite3 module – This is included in Python’s standard library, so no additional installation is needed.
* You don't need to check if the database exists - SQLite will automatically create the database file if it doesn’t exist when you call sqlite3.connect("weather_data.db")
* You DO need to check if tables exist before inserting data, by using CREATE TABLE IF NOT EXISTS in an SQL script

### Development - storing data

In [3]:
# get some data to play around with
from src.get_data import historical_data
df_historical = historical_data()
print(df_historical)

         date rainfall_mm
0  2025-01-01           0
3  2025-01-02           0
6  2025-01-03         4.8
9  2025-01-04         0.2
12 2025-01-05           0
..        ...         ...
47 2025-03-16           0
50 2025-03-17           0
53 2025-03-18           0
56 2025-03-19           0
59 2025-03-20           0

[79 rows x 2 columns]


In [4]:
from src.get_data import forecast_data
df_forecast = forecast_data()
print(df_forecast)

  date_forecast_was_made date_forecast_applies_to  rain_chance  rain_mm_low  \
0             2025-03-20               2025-03-20          0.1            0   
1             2025-03-20               2025-03-21          0.5            0   
2             2025-03-20               2025-03-22          0.4            0   
3             2025-03-20               2025-03-23          0.5            0   
4             2025-03-20               2025-03-24          0.4            0   
5             2025-03-20               2025-03-25          0.4            0   
6             2025-03-20               2025-03-26          0.5            0   

   rain_mm_high  
0             0  
1             3  
2             1  
3             3  
4             1  
5             3  
6             5  


In [ ]:
# Connect to (or create) the database
db_path = "../data/rainfall.db"
with sqlite3.connect(db_path) as conn: # automatically runs conn.close()
    cur = conn.cursor() # cursor object to interact with the database

    # Create tables if they don't exist
    cur.execute("""
    CREATE TABLE IF NOT EXISTS forecast (
        date_forecast_was_made TEXT,
        date_forecast_applies_to TEXT,
        rain_chance REAL,
        rain_mm_low REAL,
        rain_mm_high REAL,
        PRIMARY KEY (date_forecast_was_made, date_forecast_applies_to)
    );
    """)
    conn.commit()

    # Add data from the dataframe to the database
    # df_forecast.to_sql("forecast", conn, if_exists="append", index=False) # unfortunately is not idempotent so could fail due to primary key constraint

    for _, row in df_forecast.iterrows():
        cur.execute("""
        INSERT OR REPLACE INTO forecast (
            date_forecast_was_made, date_forecast_applies_to, rain_chance, rain_mm_low, rain_mm_high
        ) VALUES (?, ?, ?, ?, ?)
        """, (row['date_forecast_was_made'].strftime('%Y-%m-%d'), row['date_forecast_applies_to'].strftime('%Y-%m-%d'), row['rain_chance'], row['rain_mm_low'], row['rain_mm_high']))
        # note - initially used str(row['date_forecast_was_made']) which gives TEXT values like '2025-03-20 00:00:00'
        # changed to row['date_forecast_was_made'].strftime('%Y-%m-%d') to remove the 00:00:00 to make date filtering work better
    conn.commit()

    # Verify that the data was inserted correctly by selecting the first 3 rows
    cur.execute("SELECT * FROM forecast")
    rows = cur.fetchall()
    for row in rows:
        print(row)

('2025-03-19 00:00:00', '2025-03-19 00:00:00', 0.05, 0.0, 0.0)
('2025-03-19 00:00:00', '2025-03-20 00:00:00', 0.1, 0.0, 0.0)
('2025-03-19 00:00:00', '2025-03-21 00:00:00', 0.5, 0.0, 3.0)
('2025-03-19 00:00:00', '2025-03-22 00:00:00', 0.4, 0.0, 1.0)
('2025-03-19 00:00:00', '2025-03-23 00:00:00', 0.6, 0.0, 5.0)
('2025-03-19 00:00:00', '2025-03-24 00:00:00', 0.5, 0.0, 2.0)
('2025-03-19 00:00:00', '2025-03-25 00:00:00', 0.5, 0.0, 5.0)
('2025-03-20 00:00:00', '2025-03-20 00:00:00', 0.1, 0.0, 0.0)
('2025-03-20 00:00:00', '2025-03-21 00:00:00', 0.5, 0.0, 3.0)
('2025-03-20 00:00:00', '2025-03-22 00:00:00', 0.4, 0.0, 1.0)
('2025-03-20 00:00:00', '2025-03-23 00:00:00', 0.5, 0.0, 3.0)
('2025-03-20 00:00:00', '2025-03-24 00:00:00', 0.4, 0.0, 1.0)
('2025-03-20 00:00:00', '2025-03-25 00:00:00', 0.4, 0.0, 3.0)
('2025-03-20 00:00:00', '2025-03-26 00:00:00', 0.5, 0.0, 5.0)
('2025-03-20', '2025-03-20', 0.1, 0.0, 0.0)
('2025-03-20', '2025-03-21', 0.5, 0.0, 3.0)
('2025-03-20', '2025-03-22', 0.4, 0.0, 1.0)

### Development - retrieving data

In [6]:
with sqlite3.connect(db_path) as conn:
    df_check = pd.read_sql("SELECT * FROM forecast", conn)

# looks close but not quite the same
print(df_forecast)
print(df_check)

# confirm data types aren't the same
print(df_forecast.dtypes)
print(df_check.dtypes)

# convert them back
df_check['date_forecast_was_made'] = pd.to_datetime(df_check['date_forecast_was_made'])
df_check['date_forecast_applies_to'] = pd.to_datetime(df_check['date_forecast_applies_to'])
print(df_check)
print(df_check.dtypes)

  date_forecast_was_made date_forecast_applies_to  rain_chance  rain_mm_low  \
0             2025-03-20               2025-03-20          0.1            0   
1             2025-03-20               2025-03-21          0.5            0   
2             2025-03-20               2025-03-22          0.4            0   
3             2025-03-20               2025-03-23          0.5            0   
4             2025-03-20               2025-03-24          0.4            0   
5             2025-03-20               2025-03-25          0.4            0   
6             2025-03-20               2025-03-26          0.5            0   

   rain_mm_high  
0             0  
1             3  
2             1  
3             3  
4             1  
5             3  
6             5  
   date_forecast_was_made date_forecast_applies_to  rain_chance  rain_mm_low  \
0     2025-03-19 00:00:00      2025-03-19 00:00:00         0.05          0.0   
1     2025-03-19 00:00:00      2025-03-20 00:00:00         0.10

### Functions for storing and retrieving data

#### Create tables

In [9]:
db_path = "../data/rainfall.db"

def create_forecast_table(db_path):
    '''Creates the forecast table'''

    with sqlite3.connect(db_path) as conn:
        cur = conn.cursor()

        # Create table if it doesn't already exist
        cur.execute("""
        CREATE TABLE IF NOT EXISTS forecast (
            date_forecast_was_made TEXT,
            date_forecast_applies_to TEXT,
            rain_chance REAL,
            rain_mm_low REAL,
            rain_mm_high REAL,
            PRIMARY KEY (date_forecast_was_made, date_forecast_applies_to)
        );
        """)
        conn.commit()

def create_historical_table(db_path):
    '''Create the historical table
    
    Note: given this is a toy example it could be run every time (to avoid needing to manually create the table)
    but it only needs to be run once when the database is first set up, so despite CREATE TABLE IF NOT EXISTS
    being relatively efficient if it already exists, for best practice, I will split it into a separate function'''

    with sqlite3.connect(db_path) as conn:
        cur = conn.cursor()

        # Create table if it doesn't already exist
        cur.execute("""
        CREATE TABLE IF NOT EXISTS historical (
            date TEXT PRIMARY KEY,
            rainfall_mm REAL
        );
        """)
        conn.commit()

def check_tables(db_path):
    """Prints the schema of all tables in the SQLite database at db_path."""

    with sqlite3.connect(db_path) as conn:
        cur = conn.cursor()
        cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cur.fetchall()

        if not tables:
            print("No tables found in the database.")
            return
        
        schema_data = []

        for table in tables:
            table_name = table[0]
            cur.execute(f"PRAGMA table_info({table_name});")
            columns = cur.fetchall()

            for col in columns:
                schema_data.append((table_name, *col))

        df_schema = pd.DataFrame(schema_data, columns=["Table Name", "Column Index", "Column Name", "Data Type", "Allows NULL", "Default Value", "Primary Key"])
        print(df_schema)

create_forecast_table(db_path)
create_historical_table(db_path)
check_tables(db_path)

   Table Name  Column Index               Column Name Data Type  Allows NULL  \
0    forecast             0    date_forecast_was_made      TEXT            0   
1    forecast             1  date_forecast_applies_to      TEXT            0   
2    forecast             2               rain_chance      REAL            0   
3    forecast             3               rain_mm_low      REAL            0   
4    forecast             4              rain_mm_high      REAL            0   
5  historical             0                      date      TEXT            0   
6  historical             1               rainfall_mm      REAL            0   

  Default Value  Primary Key  
0          None            1  
1          None            2  
2          None            0  
3          None            0  
4          None            0  
5          None            1  
6          None            0  


#### Forecast data

In [ ]:
db_path = "../data/rainfall.db"

def add_forecast_data(db_path, df_to_add):
    '''Add data from df_to_add into the forecast table in the sqlite database at db_path'''

    with sqlite3.connect(db_path) as conn:
        cur = conn.cursor()

        # Add data from the dataframe to the database, ensuring idempotent (no duplicates, latest values are retained)
        for _, row in df_to_add.iterrows():
            cur.execute("""
            INSERT OR REPLACE INTO forecast (
                date_forecast_was_made, date_forecast_applies_to, rain_chance, rain_mm_low, rain_mm_high
            ) VALUES (?, ?, ?, ?, ?)
            """, (row['date_forecast_was_made'].strftime('%Y-%m-%d'), row['date_forecast_applies_to'].strftime('%Y-%m-%d'), row['rain_chance'], row['rain_mm_low'], row['rain_mm_high']))
        conn.commit()

def get_forecast_data(db_path, filter=""):
    '''Retrieve data from the forecast table in the sqlite database at db_path, using SELECT * FROM forecast
    
       The forecast database has primary key fields date_forecast_was_made and date_forecast_applies_to
       and values in rain_chance, rain_mm_low, rain_mm_high

    Keyword arguments
    filter -- can be used to append a WHERE condition to the SELECT query. Note that the dates are TEXT data type
              but date filtering should still work. For example:
              filter="date_forecast_was_made = '2025-03-19'") -> SELECT * FROM forecast WHERE date_forecast_was_made = '2025-03-19';
              filter="date_forecast_was_made > '2025-03-19'") -> SELECT * FROM forecast WHERE date_forecast_was_made > '2025-03-19';
    '''

    with sqlite3.connect(db_path) as conn:
        query = "SELECT * FROM forecast"
        if filter:
            query += f" WHERE {filter}"
        df = pd.read_sql(query, conn)

        # convert dates
        df['date_forecast_was_made'] = pd.to_datetime(df['date_forecast_was_made'])
        df['date_forecast_applies_to'] = pd.to_datetime(df['date_forecast_applies_to'])
        return df

In [24]:
# testing
add_forecast_data(db_path, df_forecast) # check function works

In [20]:
add_forecast_data(db_path="../data/rainfall_test.db", df_to_add=df_forecast) # try adding data to a fresh db

In [21]:
get_forecast_data(db_path) # check function works

testing - query is  SELECT * FROM forecast


,date_forecast_was_made,date_forecast_applies_to,rain_chance,rain_mm_low,rain_mm_high
0,2025-03-19,2025-03-19,0.05,0.0,0.0
1,2025-03-19,2025-03-20,0.10,0.0,0.0
2,2025-03-19,2025-03-21,0.50,0.0,3.0
3,2025-03-19,2025-03-22,0.40,0.0,1.0
4,2025-03-19,2025-03-23,0.60,0.0,5.0
5,2025-03-19,2025-03-24,0.50,0.0,2.0
6,2025-03-19,2025-03-25,0.50,0.0,5.0


In [ ]:
get_forecast_data(db_path, filter="date_forecast_was_made = '2025-03-19'") # check filtering works

testing - query is  SELECT * FROM forecast WHERE date_forecast_was_made = '2025-03-19'


,date_forecast_was_made,date_forecast_applies_to,rain_chance,rain_mm_low,rain_mm_high


In [ ]:
get_forecast_data(db_path, filter="DATE(date_forecast_was_made) = '2025-03-19'") # works if you use this approach

testing - query is  SELECT * FROM forecast WHERE DATE(date_forecast_was_made) = '2025-03-19'


,date_forecast_was_made,date_forecast_applies_to,rain_chance,rain_mm_low,rain_mm_high
0,2025-03-19,2025-03-19,0.05,0.0,0.0
1,2025-03-19,2025-03-20,0.10,0.0,0.0
2,2025-03-19,2025-03-21,0.50,0.0,3.0
3,2025-03-19,2025-03-22,0.40,0.0,1.0
4,2025-03-19,2025-03-23,0.60,0.0,5.0
5,2025-03-19,2025-03-24,0.50,0.0,2.0
6,2025-03-19,2025-03-25,0.50,0.0,5.0


In [26]:
db_path = "../data/rainfall.db"

def add_historical_data(db_path, df_to_add):
    '''Add data from df_to_add into the historical table in the sqlite database at db_path'''

    with sqlite3.connect(db_path) as conn:
        cur = conn.cursor()

        # Add data from the dataframe to the database, ensuring idempotent (no duplicates, latest values are retained)
        for _, row in df_to_add.iterrows():
            cur.execute("""
            INSERT OR REPLACE INTO historical (
                date, rainfall_mm
            ) VALUES (?, ?)
            """, (str(row['date']), str(row['rainfall_mm'])))
        conn.commit()

def get_historical_data(db_path, filter=""):
    '''Retrieve data from the historical table in the sqlite database at db_path, using SELECT * FROM historical
    
       The historical database has primary key field date and values in rainfall_mm

    Keyword arguments
    filter -- can be used to append a WHERE condition to the SELECT query. Note that the dates are TEXT data type
              but date filtering should still work. For example:
              filter="date = '2025-03-19'") -> SELECT * FROM forecast WHERE date = '2025-03-19';
              filter="date > '2025-03-19'") -> SELECT * FROM forecast WHERE date > '2025-03-19';
    '''

    with sqlite3.connect(db_path) as conn:
        query = "SELECT * FROM historical"
        if filter:
            query += f" WHERE {filter}"
        df = pd.read_sql(query, conn)

        # convert dates
        df['date'] = pd.to_datetime(df['date'])
        return df

In [16]:
get_historical_data(db_path) # check function works

,date,rainfall_mm
0,2025-01-01,0.0
1,2025-01-02,0.0
2,2025-01-03,4.8
3,2025-01-04,0.2
4,2025-01-05,0.0
...,...,...
74,2025-03-16,0.0
75,2025-03-17,0.0
76,2025-03-18,0.0
77,2025-03-19,0.0


In [15]:
add_historical_data(db_path, df_historical) # check function works

In [ ]:
get_historical_data(db_path, "date >= '2025-03-18'")

,date,rainfall_mm
0,2025-03-18,0.0
1,2025-03-19,0.0
2,2025-03-20,0.0


In [20]:
get_forecast_data(db_path, "date_forecast_was_made >= '2025-03-18'")

,date_forecast_was_made,date_forecast_applies_to,rain_chance,rain_mm_low,rain_mm_high
0,2025-03-19,2025-03-19,0.05,0.0,0.0
1,2025-03-19,2025-03-20,0.10,0.0,0.0
2,2025-03-19,2025-03-21,0.50,0.0,3.0
3,2025-03-19,2025-03-22,0.40,0.0,1.0
4,2025-03-19,2025-03-23,0.60,0.0,5.0
5,2025-03-19,2025-03-24,0.50,0.0,2.0
6,2025-03-19,2025-03-25,0.50,0.0,5.0
7,2025-03-20,2025-03-20,0.10,0.0,0.0
8,2025-03-20,2025-03-21,0.50,0.0,3.0
9,2025-03-20,2025-03-22,0.40,0.0,1.0


In [25]:
get_forecast_data(db_path, "date_forecast_was_made = '2025-03-20'")

,date_forecast_was_made,date_forecast_applies_to,rain_chance,rain_mm_low,rain_mm_high
0,2025-03-20,2025-03-20,0.1,0.0,0.0
1,2025-03-20,2025-03-21,0.5,0.0,3.0
2,2025-03-20,2025-03-22,0.4,0.0,1.0
3,2025-03-20,2025-03-23,0.5,0.0,3.0
4,2025-03-20,2025-03-24,0.4,0.0,1.0
5,2025-03-20,2025-03-25,0.4,0.0,3.0
6,2025-03-20,2025-03-26,0.5,0.0,5.0
